In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report

from amb_sdk.sdk import DarwinSdk



In [4]:
# Login
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('arathis269@gmail.com', 'ZYmhY52mHE')

if not status:
    print(msg)

In [8]:
path = 'data/'

In [13]:
dataset_name = 'MEASURESOFBIRTHANDDEATH.csv'
df = pd.read_csv(os.path.join(path, dataset_name))
df.head()

,State_FIPS_Code,County_FIPS_Code,CHSI_County_Name,CHSI_State_Name,CHSI_State_Abbr,Strata_ID_Number,LBW,LBW_Ind,Min_LBW,Max_LBW,...,CI_Max_Suicide,Injury,Injury_Ind,Min_Injury,Max_Injury,CI_Min_Injury,CI_Max_Injury,Total_Births,Total_Deaths,MOBD_Time_Span
0,1,1,Autauga,Alabama,AL,29,8.1,4,6.0,8.1,...,23.0,23.0,4.0,16.4,27.9,16.9,30.7,3048.0,1867.0,1999-2003
1,1,3,Baldwin,Alabama,AL,16,8.6,4,6.3,9.1,...,18.2,30.1,4.0,15.6,29.1,25.1,35.1,5471.0,4388.0,2001-2003
2,1,5,Barbour,Alabama,AL,51,11.0,4,6.7,11.9,...,16.9,29.6,4.0,17.5,35.2,21.5,39.7,1835.0,1538.0,1999-2003
3,1,7,Bibb,Alabama,AL,42,8.7,4,5.1,10.3,...,21.3,30.0,4.0,20.4,38.1,22.7,38.8,2801.0,2012.0,1994-2003
4,1,9,Blount,Alabama,AL,28,7.6,4,5.2,9.2,...,19.8,24.4,4.0,16.4,32.6,18.7,31.2,3318.0,2453.0,1999-2003


In [14]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(path, dataset_name))
if not status:
    print(dataset)

In [16]:
# clean dataset
target = "CHSI_State_Name"
status, job_id = ds.clean_data(dataset_name, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-09T19:41:39.281319', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['MEASURESOFBIRTHANDDEATH.csv'], 'artifact_names': ['53bed972daf5431aa3d8dafb20ba48df'], 'model_name': None, 'job_error': None}
{'status': 'Complete', 'starttime': '2019-04-09T19:41:39.281319', 'endtime': '2019-04-09T19:41:48.170744', 'percent_complete': 100, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['MEASURESOFBIRTHANDDEATH.csv'], 'artifact_names': ['53bed972daf5431aa3d8dafb20ba48df'], 'model_name': None, 'job_error': ''}


In [19]:
model = target + "_model0"
status, job_id = ds.create_model(dataset_names = dataset_name, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

400: BAD REQUEST - {"message": "A model with this name already exists"}



In [20]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

404: NOT FOUND - {"message": "Failed to find a dataset that the model was trained on. You have requested this URI [/v1/analyze/model/CHSI_State_Name_model0] but did you mean /v1/analyze/model/<model_name> ?"}



TypeError: string indices must be integers